In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import pandas as pd
from ultralytics import YOLO
import cv2
from matplotlib import pyplot as plt
import torch
from tqdm import tqdm 
import tensorflow as tf

In [2]:
# Load the model
model = YOLO('yolov8x-pose.pt')
n_timesteps = 30

In [3]:
df_fall = pd.read_csv('urfall-cam0-falls.csv', header=None)
df_adl = pd.read_csv('urfall-cam0-adls.csv', header=None)
x = np.empty((0,n_timesteps,17,3),float)
y = np.empty((0,n_timesteps),float)

In [4]:

def create_dataset(x,y,df):
    batch_x = np.zeros((30,17,3),float)
    batch_y = np.zeros((30),float)
    last_path = ''
    frame = 0

    for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing Frames"):
        path = 'data/'+row[0]+'-cam0-rgb'
        img_path = row[0]+'-cam0-rgb-'+'0'*(3-len(str(row[1])))+str(row[1])+'.png'
        img = cv2.imread(path+'/'+img_path)
        if row[2] == 1 and row[0][:3] != 'adl':
            label = 1
        else:
            label = 0
        label = np.array([label])
        # Get the prediction
        results = model(img, device='0', verbose=False)
        keypoints = results[0].keypoints
        if keypoints.conf is None:
            data = np.zeros((1,17,3))
        else:
            keypoints = keypoints.cpu().numpy() if torch.cuda.is_available() else keypoints.numpy()
            data = np.concatenate((np.expand_dims(keypoints.conf, axis=-1),keypoints.xyn),axis=2)
        if data.shape[0] > 1:
            data = np.expand_dims(data[0],axis=0)
        if last_path == path or last_path == '':
                batch_x = np.append(batch_x[1:],data,axis=0)
                batch_y = np.append(batch_y[1:],label,axis=0)
                x = np.append(x,np.expand_dims(batch_x,axis=0),axis=0)
                y = np.append(y,np.expand_dims(batch_y,axis=0),axis=0)
        else:
            frame = 0
            batch_x = np.zeros((30,17,3),float)
            batch_y = np.zeros((30),float)
            batch_x = np.append(batch_x[1:],data,axis=0)
            batch_y = np.append(batch_y[1:],label,axis=0)
        frame += 1
        last_path = path
    print(x.shape,y.shape)
    return x,y

In [5]:
x, y = create_dataset(x,y,df_fall)
x, y = create_dataset(x,y,df_adl)
x.shape, y.shape
np.save(r'data_csv\x.npy',x)
np.save(r'data_csv\y.npy',y)

Processing Frames: 100%|██████████| 2995/2995 [01:47<00:00, 27.82it/s]


(2966, 30, 17, 3) (2966, 30)


Processing Frames: 100%|██████████| 8549/8549 [06:51<00:00, 20.78it/s]


(11476, 30, 17, 3) (11476, 30)


In [6]:
print("X has NaN:", np.isnan(x).any())
print("X has Inf:", np.isinf(x).any())
for i in range(x.shape[0]):  # Duyệt qua chiều đầu tiên
    for j in range(x.shape[1]):  # Duyệt qua chiều thứ hai
        for k in range(x.shape[2]):  # Duyệt qua chiều thứ ba
            for l in range(x.shape[3]):  # Duyệt qua chiều thứ tư
                value = x[i, j, k, l]
                # Thực hiện một hành động với 'value'
                # Ví dụ: kiểm tra nếu giá trị lớn hơn 1
                if value > 1:
                    print(f"Element at index ({i}, {j}, {k}, {l}) is greater than 1: {value}")


X has NaN: False
X has Inf: False


In [2]:
def split_into_train_val(x,y,train_size):
    indices = np.arange(x.shape[0])
    np.random.shuffle(indices)
    x = x[indices]
    y = y[indices]
    train_size = int(x.shape[0]*train_size)
    x_train = x[:train_size]
    y_train = y[:train_size]
    x_val = x[train_size:]
    y_val = y[train_size:]
    return x_train, y_train, x_val, y_val

In [3]:
def ohe_y(y):
    target = np.zeros((y.shape[0],2),dtype='int')
    for row in range(y.shape[0]):
        if y[row][14] == 0:
            col = 0
        else:
            col = 1
        target[row,col] = 1
    return target
    

In [4]:
x = np.load(r'data_csv\x.npy')
y = np.load(r'data_csv\y.npy')
print(x.shape, y.shape)
print(type(y),y.shape)
y = ohe_y(y)
x_train, y_train, x_val, y_val = split_into_train_val(x,y,0.8)
print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)
np.save(os.path.join('data_csv','x_val.npy'),x_val)
np.save(os.path.join('data_csv','y_val.npy'),y_val)
np.save(os.path.join('data_csv','x_train.npy'),x_train)
np.save(os.path.join('data_csv','y_train.npy'),y_train)
print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)


(11476, 30, 17, 3) (11476, 30)
<class 'numpy.ndarray'> (11476, 30)
(9180, 30, 17, 3) (9180, 2) (2296, 30, 17, 3) (2296, 2)
(9180, 30, 17, 3) (9180, 2) (2296, 30, 17, 3) (2296, 2)


In [5]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D , MaxPooling1D , LSTM, TimeDistributed, Flatten, Dropout, Input
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

In [6]:
print("X has NaN:", np.isnan(x_train).any())
print("X has Inf:", np.isinf(x_train).any())
# Kiểm tra chi so max trong x
print("X max:", np.max(x_train))


X has NaN: False
X has Inf: False
X max: 1.0


In [7]:
y_train = np.load(r'data_csv\y_train.npy')
y_val = np.load(r'data_csv\y_val.npy')
x_train = np.load(r'data_csv\x_train.npy')
x_val = np.load(r'data_csv\x_val.npy')
x_train = x_train.astype('float32')
y_train = y_train.astype('float32')
x_val = x_val.astype('float32')
y_val = y_val.astype('float32')
n_steps = x_train.shape[1]
n_steps
model = Sequential()
model.add(Input(shape=(n_steps, 17, 3)))
model.add(TimeDistributed(Conv1D(filters=16, kernel_size=2, activation="relu")))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
# the system uses a TimeDistributed wrapper which aligns the output of the flattened array of CNN sequentially
model.add(TimeDistributed(Flatten()))
model.add(LSTM(128))
model.add(Dense(32))
model.add(Dense(2, activation="softmax"))
optimizer = Adam(learning_rate=1e-5)
model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])

In [8]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed                │ (None, 30, 16, 16)     │           112 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 30, 8, 16)      │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 30, 128)        │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 135,890 (530.82 KB)

 Trainable params: 135,890 (530.82 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
def save_model(model, model_name):
    model.save(model_name)
    print(f"Model saved as {model_name}")

early_stopping = EarlyStopping(
    monitor='accuracy',
    min_delta=0.00005,
    patience=10,
    verbose=1,
    restore_best_weights=True,
)
class NaNCallback(tf.keras.callbacks.Callback):
    def on_batch_end(self, batch, logs=None):
        if any(tf.math.is_nan(logs[key]) for key in logs):
            print(f"NaN detected in batch {batch}, stopping training")
            self.model.stop_training = True

nan_callback = NaNCallback()
# callback to adjust learning rate on a plateau
lr_scheduler = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.1,
    patience=20,
    min_lr=1e-7,
    verbose=1,
)

callbacks = [
    early_stopping,
    lr_scheduler,
    nan_callback,
]

In [10]:
num_epochs = 300
history = model.fit(x_train,y_train,epochs=num_epochs, validation_data=(x_val,y_val),verbose=1,callbacks=callbacks)

Epoch 1/300
287/287 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - accuracy: 0.9377 - loss: 0.4587 - val_accuracy: 0.9560 - val_loss: 0.2027 - learning_rate: 1.0000e-05
Epoch 2/300
287/287 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9593 - loss: 0.1816 - val_accuracy: 0.9560 - val_loss: 0.1810 - learning_rate: 1.0000e-05
Epoch 3/300
287/287 ━━━━━━━━━━━━━━━━━━━━ 8s 26ms/step - accuracy: 0.9586 - loss: 0.1696 - val_accuracy: 0.9560 - val_loss: 0.1700 - learning_rate: 1.0000e-05
Epoch 4/300
287/287 ━━━━━━━━━━━━━━━━━━━━ 10s 33ms/step - accuracy: 0.9567 - loss: 0.1546 - val_accuracy: 0.9560 - val_loss: 0.1512 - learning_rate: 1.0000e-05
Epoch 5/300
287/287 ━━━━━━━━━━━━━━━━━━━━ 10s 35ms/step - accuracy: 0.9588 - loss: 0.1336 - val_accuracy: 0.9660 - val_loss: 0.1068 - learning_rate: 1.0000e-05
Epoch 6/300
287/287 ━━━━━━━━━━━━━━━━━━━━ 10s 34ms/step - accuracy: 0.9695 - loss: 0.0882 - val_accuracy: 0.9734 - val_loss: 0.0663 - learning_rate: 1.0000e-05
Epoch 7/300
287/287 ━━━━━━━━━━━━━━━━━━━━ 10s 34m

In [16]:
model.save('model.h5')

: 